## Ann Arbor Permit Cleaning

First we'll import the previous file from Dr Brooks extra credit assignment in SIADS 505 (df1). We'll use this as a guideline for the cleaning of the commerical (df2) and residential (df3) permits we downloaded before merging all of them together (df) for separate analyses. 

In [ ]:
import pandas as pd

C:\Users\Owner\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
files = [
    'AA_solar_permits_Building_Commercial.csv',
    'AA_solar_permits_Electrical_Commercial.csv',
    ]

dataframes = []

for file in files:
    df = pd.read_csv(file)
    dataframes.append(df)

df = pd.concat(dataframes, ignore_index=True)

In [ ]:
df.to_csv('Ann_Arbor_Permits_Commercial.csv')

Not all of the columns match, so we'll fix that before merging. Now we need to check if it is the same with the residential permits.

In [ ]:
files = [
    'AA_solar_permits_Building_Residential_ground.csv',
    'AA_solar_permits_Electrical_Residential_ground.csv',
    'AA_solar_permits_Electrical_Residential_roof.csv',
    'AA_solar_permits_Electrical_Residential_roof (1).csv',
    'AA_solar_permits_Electrical_Residential_roof (2).csv',
    'AA_solar_permits_Electrical_Residential_roof (3).csv',
    'AA_solar_permits_Electrical_Residential_roof (4).csv',
    'AA_solar_permits_Electrical_Residential_roof (5).csv',
    'AA_solar_permits_Electrical_Residential_roof (6).csv',
    'AA_solar_permits_Electrical_Residential_roof (7).csv'
]

dataframes = []

for file in files:
    df = pd.read_csv(file)
    dataframes.append(df)

df2 = pd.concat(dataframes, ignore_index=True)

In [ ]:
df2.to_csv('Ann_Arbor_Permits_Residential.csv')

In [ ]:
res_and_comm = pd.concat([df, df2], ignore_index=True)

Looks like we can drop a few columns from df2&df3, and rename some in df1 to make the concatenation easier. df1 also has the address broken into individual lines, to obtain different information, so we'll correct that in df2&df3 as well.

In [ ]:
df.columns = ['Type', 'Case Workclass', 'Status', 'Issued Date', 'Applied Date',
       'Expiration Date', 'Finalized Date', 'AddressLine1', 'AddressLine2',
       'AddressLine3', 'PostalCode', 'Description']

res_and_comm.drop(['Case Number', 'Project Name', 'Module Name', 'Main Parcel'], axis=1, inplace=True)

In [ ]:
print(df.columns)
print(res_and_comm.columns)

In [ ]:
df.sample(3)

In [19]:
import re

res_and_comm['PostalCode'] = res_and_comm['Address'].str.extract(r'(\d{5})$')
res_and_comm['AddressLine1'] = res_and_comm['Address'].str.extract(r'^(.*?)(?=\s+Ann Arbor\s+MI\s+\d{5}$)')
res_and_comm['AddressLine2'] = 'Ann Arbor'
res_and_comm['AddressLine3'] = 'MI'

# 5) Optional: strip whitespace
for col in ['AddressLine1','AddressLine2','AddressLine3','PostalCode']:
    res_and_comm[col] = res_and_comm[col].astype(str).str.strip()
    
res_and_comm.drop(columns=['Address'], inplace=True)

In [ ]:
df.head(1)

In [ ]:
res_and_comm.head()

In [21]:
res_and_comm['Case Workclass'] = 'Solar'

# Define the new order of columns
new_column_order = ['Type', 'Case Workclass', 'Status', 'Issued Date', 'Applied Date', 
                    'Expiration Date', 'Finalized Date', 'AddressLine1', 'AddressLine2', 
                    'AddressLine3', 'PostalCode', 'Description']

# Reorder the columns
rc_reordered = res_and_comm[new_column_order]

In [ ]:
df = pd.concat([df, rc_reordered], ignore_index=True)

In [ ]:
# save all modified df to csv
# apply datetime to all date columns
col_list = ['Applied Date', 'Issued Date', 'Expiration Date', 'Finalized Date']
for col in col_list:
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [27]:
df.to_csv('All_Ann_Arbor_Permits.csv')